In [1]:
def find_wav_files(folder_path):
    wav_files = []

    # Walk through the directory tree rooted at folder_path
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".wav"):
                wav_files.append(os.path.abspath(os.path.join(root, file)))

    return wav_files

In [2]:
!python3 --version

Python 3.10.0


In [3]:
import os

In [4]:
# get path to the IEMOCAP dataset
IEMOCAP_filepath = f"/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP"
IEMOCAP_audio = find_wav_files(IEMOCAP_filepath)

In [5]:
print(IEMOCAP_audio[:5])
print(len(IEMOCAP_audio))

['/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Audio/Session4/Ses04M_script02_2/Ses04M_script02_2_M042.wav', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Audio/Session4/Ses04M_script02_2/Ses04M_script02_2_M040.wav', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Audio/Session4/Ses04M_script02_2/Ses04M_script02_2_M041.wav', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Audio/Session4/Ses04M_script02_2/Ses04M_script02_2_F015.wav', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Audio/Session4/Ses04M_script02_2/Ses04M_script02_2_F001.wav']
10039


In [6]:
import os

def find_files(path):
    file_list = []

    for root, dirs, files in os.walk(path):
        for file in files:
            if not file.endswith("cat.txt") and not file.startswith("."):
                file_list.append(os.path.join(root, file))

    return file_list

# Replace folder_path with the path you want to traverse
folder_path = "/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Classification"
file_paths = find_files(folder_path)
print(len(file_paths), file_paths[:5])

151 ['/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Classification/Session4/Ses04F_script02_1.txt', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Classification/Session4/Ses04M_script03_2.txt', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Classification/Session4/Ses04M_script01_1.txt', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Classification/Session4/Ses04F_script02_2.txt', '/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/Classification/Session4/Ses04M_script01_3.txt']


In [7]:
import re
import json

def parse_text_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = []
    current_entry = None

    for line in lines:
        line = line.strip()

        # Check if the line matches the format for entries
        if re.match(r'\[\d+\.\d+ - \d+\.\d+\]\s+\S+\s+\S+\s+\[\d+\.\d+, \d+\.\d+, \d+\.\d+\]', line):
            if current_entry:
                data.append(current_entry)
            current_entry = {}
            parts = re.split(r'[\[\] -]', line)
            current_entry['START_TIME'] = float(parts[1])
            current_entry['END_TIME'] = parts[3]
            current_entry['TURN_NAME'] = parts[4].strip()
            current_entry['EMOTION'] = parts[5].strip()
            va_list = re.findall(r'\d+\.\d+', parts[6])
            if len(va_list) >= 3:
                current_entry['VALENCE'] = float(va_list[0])
                current_entry['AROUSAL'] = float(va_list[1])
                current_entry['DOMINANCE'] = float(va_list[2])
        elif current_entry and ':' in line:
            parts = re.split(r':\s+', line)
            current_entry[parts[0]] = parts[1]

    if current_entry:
        data.append(current_entry)

    return data

def get_emotions(filepath):
  data = parse_text_file(filepath)
  entries = []
  for d in data:
    entry = [
      d["EMOTION"].split()[0],
      [d.get("C-E1", "").split(";")[0],
       d.get("C-E2", "").split(";")[0],
       d.get("C-E3", "").split(";")[0],
       d.get("C-E4", "").split(";")[0]]
    ]
    # get rid of empty strings
    entry[1] = [item for item in entry[1] if item != ""]
    #remove duplicated
    entry[1] = list(set(entry[1]))
    entries.append(entry)
  return entries

In [8]:
results = []
for index, file in zip(range(len(file_paths)),file_paths):
  results.extend(get_emotions(file))
print(len(results), results[:5])

10039 [['Ses04F_script02_1_F000', ['Neutral', 'Sadness']], ['Ses04F_script02_1_F001', ['Frustration', 'Anger']], ['Ses04F_script02_1_F002', ['Frustration', 'Anger']], ['Ses04F_script02_1_F003', ['Frustration', 'Anger']], ['Ses04F_script02_1_F004', ['Frustration', 'Anger']]]


In [9]:
# wav file paths IEMOCAP_audio
# txt file paths file_paths
# emotion classification filename_emotion
# make a list with filename, emotion and absolute filepath (around 25s runtime, must perform 100,781,521 operations xd)
IEMOCAP_to_df = []
for i in range(len(results)):
  for j in range(len(results)):
    if IEMOCAP_audio[i][:-4].split("/")[-1] == results[j][0]:
      IEMOCAP_to_df.append([results[j][0], IEMOCAP_audio[i], results[j][1]])

In [10]:
# print if there are any mismatching names and files
for entry in zip(range(len(results)), IEMOCAP_to_df):
  print(entry[0]) if (entry[1][0] != entry[1][1][:-4].split("/")[-1]) else ""

In [11]:
# make a df with filename, emotion, absolute file path
import pandas as pd
import os
df = pd.DataFrame(IEMOCAP_to_df, columns=["filename", "abs_filepath", "emotion"])
df

,filename,abs_filepath,emotion
0,Ses04M_script02_2_M042,/Users/nicokalkusinski/Documents/Github repos/...,"[Happiness, Frustration, Neutral]"
1,Ses04M_script02_2_M040,/Users/nicokalkusinski/Documents/Github repos/...,"[Happiness, Neutral]"
2,Ses04M_script02_2_M041,/Users/nicokalkusinski/Documents/Github repos/...,"[Happiness, Neutral]"
3,Ses04M_script02_2_F015,/Users/nicokalkusinski/Documents/Github repos/...,"[Frustration, Neutral, Sadness]"
4,Ses04M_script02_2_F001,/Users/nicokalkusinski/Documents/Github repos/...,"[Frustration, Other, Anger]"
...,...,...,...
10034,Ses01F_script02_2_M010,/Users/nicokalkusinski/Documents/Github repos/...,"[Frustration, Neutral]"
10035,Ses01F_script02_2_M004,/Users/nicokalkusinski/Documents/Github repos/...,"[Frustration, Neutral]"
10036,Ses01F_script02_2_F019,/Users/nicokalkusinski/Documents/Github repos/...,"[Happiness, Other]"
10037,Ses01F_script02_2_F031,/Users/nicokalkusinski/Documents/Github repos/...,"[Frustration, Disgust, Anger]"


In [30]:
files['emotion'] = df['emotion'].apply(lambda x: ' '.join(char for char in x if char not in "[',']"))
df

,filename,abs_filepath,emotion
0,Ses04M_script02_2_M042,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Frustration Neutral
1,Ses04M_script02_2_M040,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral
2,Ses04M_script02_2_M041,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral
3,Ses04M_script02_2_F015,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Sadness Neutral
4,Ses04M_script02_2_F001,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Other
...,...,...,...
10034,Ses01F_script02_2_M010,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral
10035,Ses01F_script02_2_M004,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral
10036,Ses01F_script02_2_F019,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Other
10037,Ses01F_script02_2_F031,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Disgust


In [31]:
df.to_csv("/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/iemocap.csv", index=False)

In [3]:
import pandas as pd

In [4]:
!python3.10 -m pip install tqdm

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [10]:
!pip show tqdm

In [2]:
import os
import pandas as pd
from tqdm import tqdm
files = pd.read_csv("/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/iemocap.csv")
files.rename(columns={'emotion':'emotions_list'}, inplace = True)
files

,filename,abs_filepath,emotions_list
0,Ses04M_script02_2_M042,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Frustration Neutral
1,Ses04M_script02_2_M040,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral
2,Ses04M_script02_2_M041,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral
3,Ses04M_script02_2_F015,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Sadness Neutral
4,Ses04M_script02_2_F001,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Other
...,...,...,...
10034,Ses01F_script02_2_M010,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral
10035,Ses01F_script02_2_M004,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral
10036,Ses01F_script02_2_F019,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Other
10037,Ses01F_script02_2_F031,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Disgust


In [3]:
ground_truth_files = pd.read_csv("/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/IEMOCAP_files_single_emotion.csv")
ground_truth_files

,filename,emotion,abs_filepath
0,Ses04M_script02_2_F014,xxx,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
1,Ses04M_script02_2_F019,sad,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
2,Ses04M_script02_2_F034,xxx,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
3,Ses04M_script02_2_M030,xxx,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
4,Ses04M_script02_2_F018,sad,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
...,...,...,...
10034,Ses01F_script02_2_M043,neu,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
10035,Ses01F_script02_2_F041,neu,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
10036,Ses01F_script02_2_M009,fru,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...
10037,Ses01F_script02_2_F012,fru,/content/drive/MyDrive/FYP/Datasets/IEMOCAP/Au...


In [4]:
df = pd.merge(files, ground_truth_files, on='filename', how='inner')
df = df.drop(columns='abs_filepath_y')
df

,filename,abs_filepath_x,emotions_list,emotion
0,Ses04M_script02_2_M042,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Frustration Neutral,xxx
1,Ses04M_script02_2_M040,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral,hap
2,Ses04M_script02_2_M041,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral,hap
3,Ses04M_script02_2_F015,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Sadness Neutral,xxx
4,Ses04M_script02_2_F001,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Other,xxx
...,...,...,...,...
10034,Ses01F_script02_2_M010,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral,fru
10035,Ses01F_script02_2_M004,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral,fru
10036,Ses01F_script02_2_F019,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Other,hap
10037,Ses01F_script02_2_F031,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Disgust,xxx


In [5]:
e_encoder = {
    'ang':'Anger',
    'exc':'Excited',
    'fea':'Fear',
    'fru':'Frustration',
    'hap':'Happiness',
    'neu':'Neutral',
    'oth':'Other',
    'sad':'Sadness',
    'sur':'Surprise',
    'dis':'Disgust',
    'xxx':'Unclassified'
}

df.emotion = df.emotion.apply(lambda emotion: e_encoder[emotion])
df

,filename,abs_filepath_x,emotions_list,emotion
0,Ses04M_script02_2_M042,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Frustration Neutral,Unclassified
1,Ses04M_script02_2_M040,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral,Happines
2,Ses04M_script02_2_M041,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral,Happines
3,Ses04M_script02_2_F015,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Sadness Neutral,Unclassified
4,Ses04M_script02_2_F001,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Other,Unclassified
...,...,...,...,...
10034,Ses01F_script02_2_M010,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral,Frustration
10035,Ses01F_script02_2_M004,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral,Frustration
10036,Ses01F_script02_2_F019,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Other,Happines
10037,Ses01F_script02_2_F031,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Disgust,Unclassified


In [6]:
df.rename(columns={'emotions_list':'multi_emotion',
                   'emotion':'single_emotion',
                   'abs_filepath_x':'filepath'},
          inplace = True)
df

,filename,filepath,multi_emotion,single_emotion
0,Ses04M_script02_2_M042,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Frustration Neutral,Unclassified
1,Ses04M_script02_2_M040,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral,Happines
2,Ses04M_script02_2_M041,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Neutral,Happines
3,Ses04M_script02_2_F015,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Sadness Neutral,Unclassified
4,Ses04M_script02_2_F001,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Other,Unclassified
...,...,...,...,...
10034,Ses01F_script02_2_M010,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral,Frustration
10035,Ses01F_script02_2_M004,/Users/nicokalkusinski/Documents/Github repos/...,Frustration Neutral,Frustration
10036,Ses01F_script02_2_F019,/Users/nicokalkusinski/Documents/Github repos/...,Happiness Other,Happines
10037,Ses01F_script02_2_F031,/Users/nicokalkusinski/Documents/Github repos/...,Anger Frustration Disgust,Unclassified


In [7]:
### FOLLOWING CODE WAS PASTED FROM https://www.kaggle.com/code/mostafaabdlhamed/speech-emotion-recognition-97-25-accuracy/notebook
import librosa
import numpy as np
# Data manipulation functions
# add a random noise to the audio
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# stretch the audio by 20%
def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(y=data, rate=rate)

# shift the audio
def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

# change the pitch of the audio
def pitch(data, sampling_rate, n_steps=0.7):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=n_steps)


# Feature extraction function
# zero crossing rate
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)

# root mean square error
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)

# Mel-frequency cepstral coefficients
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T) if not flatten else np.ravel(mfcc.T)


# extract features from the file
def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result = np.array([])
    result = np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

# perform data augmentation on a file and extract features from it
def get_features(path, duration=2):
    # get the file
    data,sr = librosa.load(path)
    #extract features from an original file
    aud = extract_features(data)
    audio = np.array(aud)

    #add noise and extract features
    noised_audio = noise(data)
    aud2 = extract_features(noised_audio)
    audio = np.vstack((audio,aud2))

    #add pitch and extract features
    pitched_audio = pitch(data,sr)
    aud3 = extract_features(pitched_audio)
    audio = np.vstack((audio,aud3))

    #add pitch and noise and extract features
    # pitched_audio1 = pitch(data,sr)
    # pitched_noised_audio = noise(pitched_audio1)
    # aud4 = extract_features(pitched_noised_audio)
    # audio = np.vstack((audio, aud4))

    #add shift and extract features
    # shifted_audio = shift(data)
    # aud5 = extract_features(shifted_audio)
    # audio = np.vstack((audio, aud5))

    # #add stretch and extract features
    # stretched_audio = stretch(data)
    # aud6 = extract_features(shifted_audio)
    # audio = np.vstack((audio, aud6))

    return audio

In [9]:
X, y, z = [], [], []

In [11]:
from tqdm import tqdm
import timeit

print("Feature extraction")
start = timeit.default_timer()
for path, m_emotion, s_emotion, index in zip(df.filepath, df.multi_emotion, df.single_emotion, tqdm(range(df.shape[0]))):
    features = get_features(path)
    for i in features:
        X.append(i)
        y.append(s_emotion)
        z.append(m_emotion)
print("done")
stop = timeit.default_timer()
print("Time: ", stop - start)

Feature extraction


100%|████████████████████████████████████▉| 10038/10039 [14:21<00:00, 11.65it/s]

done
Time:  861.3270420830231


In [13]:
emotions = pd.DataFrame(X)
emotions['single_emotion'] = y
emotions['multi_emotion'] = z
emotions.head()

,0,1,2,3,4,5,6,7,8,9,...,32354,32355,32356,32357,32358,32359,32360,32361,single_emotion,multi_emotion
0,0.020020,0.026367,0.028809,0.029297,0.029785,0.026367,0.027832,0.018066,0.017090,0.023926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Happiness Frustration Neutral
1,0.163086,0.223145,0.272461,0.276367,0.258301,0.245117,0.263184,0.231445,0.245117,0.268555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Happiness Frustration Neutral
2,0.023926,0.031738,0.037109,0.029297,0.029297,0.025879,0.025879,0.027344,0.022949,0.026855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Happiness Frustration Neutral
3,0.032227,0.052246,0.070312,0.062988,0.060547,0.051758,0.046387,0.043457,0.038086,0.033203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Happines,Happiness Neutral
4,0.182617,0.290527,0.403809,0.433105,0.451172,0.449707,0.453613,0.459473,0.444824,0.446289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Happines,Happiness Neutral


In [18]:
emotions.single_emotion = emotions.single_emotion.apply(lambda emotion: "Happiness" if emotion == "Happines" else emotion)

,0,1,2,3,4,5,6,7,8,9,...,32354,32355,32356,32357,32358,32359,32360,32361,single_emotion,multi_emotion
0,0.020020,0.026367,0.028809,0.029297,0.029785,0.026367,0.027832,0.018066,0.017090,0.023926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Happiness Frustration Neutral
1,0.163086,0.223145,0.272461,0.276367,0.258301,0.245117,0.263184,0.231445,0.245117,0.268555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Happiness Frustration Neutral
2,0.023926,0.031738,0.037109,0.029297,0.029297,0.025879,0.025879,0.027344,0.022949,0.026855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Happiness Frustration Neutral
3,0.032227,0.052246,0.070312,0.062988,0.060547,0.051758,0.046387,0.043457,0.038086,0.033203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Happiness,Happiness Neutral
4,0.182617,0.290527,0.403809,0.433105,0.451172,0.449707,0.453613,0.459473,0.444824,0.446289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Happiness,Happiness Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30490,0.190918,0.283691,0.386230,0.387695,0.396973,0.406250,0.413086,0.419434,0.423340,0.419434,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Anger Frustration Disgust
30491,0.015625,0.029785,0.041016,0.055176,0.068848,0.075684,0.080566,0.071289,0.060547,0.048828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unclassified,Anger Frustration Disgust
30492,0.019043,0.026855,0.034668,0.039551,0.037109,0.038574,0.037109,0.042480,0.039062,0.042480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frustration,Frustration Neutral
30493,0.029785,0.046387,0.062988,0.079590,0.082031,0.081543,0.092773,0.116211,0.102539,0.101074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frustration,Frustration Neutral


In [ ]:
# one hot encoding if needed
specified_emotions = ['Neutral', 'Excited', 'Anger', 'Frustration', 'Fear', 'Disgust', 'Surprise', 'Sadness', 'Happiness', 'Other']
for emotion in specified_emotions:
    df[emotion] = df['multi_emotion'].apply(lambda x: 1 if emotion in x else 0)

In [20]:
emotions.shape

(30495, 32364)

In [21]:
emotions.to_csv("/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/iemocap_features_241123.csv")

In [2]:
#this will take time, this csv is 6.6gb 💀
import pandas as pd
from tqdm import tqdm
file_path = "/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/iemocap_final.csv"
total_lines = 60234

# Create a tqdm progress bar and use the chunksize parameter for better progress estimation
chunk_size = 1000  # Adjust the chunk size based on your file size and available memory
tqdm.pandas(desc="Reading CSV")
df = pd.read_csv(file_path, chunksize=chunk_size)

# Concatenate the chunks into a DataFrame
df = pd.concat(tqdm(df, total=total_lines // chunk_size, dynamic_ncols=True, unit="chunk"))

61chunk [07:55,  7.80s/chunk]                                                   


NameError: name 'emotions' is not defined

In [26]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,Neutral,Excited,Anger,Frustration,Fear,Disgust,Surprise,Sadness,Happiness,Other
0,0.020020,0.026367,0.028809,0.029297,0.029785,0.026367,0.027832,0.018066,0.017090,0.023926,...,1,0,0,1,0,0,0,0,1,0
1,0.173828,0.234863,0.289551,0.297852,0.273926,0.260742,0.275391,0.239746,0.250977,0.289551,...,1,0,0,1,0,0,0,0,1,0
2,0.023926,0.031738,0.037109,0.029297,0.029297,0.025879,0.025879,0.027344,0.022949,0.026855,...,1,0,0,1,0,0,0,0,1,0
3,0.043945,0.060059,0.067383,0.053711,0.051758,0.046387,0.047363,0.051758,0.048340,0.052246,...,1,0,0,1,0,0,0,0,1,0
4,0.013184,0.023438,0.028320,0.027344,0.022949,0.017578,0.021973,0.026367,0.032715,0.035645,...,1,0,0,1,0,0,0,0,1,0


In [19]:
df[specified_emotions].sum()

NameError: name 'specified_emotions' is not defined

In [10]:
df_multilabel.to_csv("/Users/nicokalkusinski/Documents/Github repos/FYP/FYP-Speech-Emotion-Recognition/Datasets/IEMOCAP/iemocap_multilabel.csv")

In [ ]:
X = df_multilabel.iloc[:, :-10].values
y = df_multilabel.iloc[:, -10:-1].values
y

In [ ]:
!python3.10 -m pip install sklearn

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42,test_size=0.2, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape